# Load Train Set

features (3681) | labels (1)

In [1]:
# import pandas as pd
import cudf
import cupy
import pandas as pd
import numpy as np

# df = pd.read_feather("dts/np_dataset_train.ftr")
df = pd.read_csv("dts/np_dataset_train.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 1.1 GB


## train/test Split

In [2]:
from sklearn.model_selection import train_test_split

rs = 2

train, test = train_test_split(df, test_size=0.2, random_state=rs)
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 31392 to 23720
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 899.4 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 2727 to 32354
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 224.9 MB


## Classifier definition

In [11]:
from random import seed
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier as tree_classifier
from scipy import stats


class random_forest():
    def __init__(self, number_of_trees=1, number_of_iteractions=1, max_depth=1, random_state=1):
        self.iteractions = number_of_iteractions
        self.number_of_trees = number_of_trees
        self.max_depth = max_depth
        self.forest = []
        self.random_state = random_state

    def bootstrap(self, x_samples):
        """ Create subsets of the input, and returns the indexes 
            from the original sample."""
        index = []

        sample_length = len(x_samples)
        np.random.RandomState(self.random_state)
        for i in range(self.number_of_trees):
            index.append(np.random.randint(0, sample_length, [
                         1, sample_length]))
        return index

    def predict(self, X):
        y = []
        for tree in self.forest:
            y.append(tree.predict(X))
        mode_ = stats.mode(y, axis=0)
        mode_ = np.transpose(mode_[0])
        return mode_

    def fit(self, x_samples, y_samples):
        idx = self.bootstrap(x_samples)

        for t in range(self.number_of_trees):
            tree = tree_classifier(
                criterion="gini", max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(x_samples[flatten(idx[t])], y_samples[flatten(idx[t])])
            self.forest.append(tree)

    # def train(self, x_samples, y_samples):
    #     self.bagging(x_samples, y_samples)


def flatten(t):
    return [item for item in t]


In [12]:
# from sklearn import svm
# from cuml.svm import SVC
# import random_forest as rf

clf = random_forest(number_of_trees=1, number_of_iteractions=1, max_depth=1)


## Sending Data to GPU

In [13]:
# x = cudf.DataFrame(train.iloc[:, :-1])
# y = cupy.array(train.iloc[:, -1])

x = pd.DataFrame(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])

print(x.shape)
print(y.shape)

(32000, 3683)
(32000,)


## Model Fitting

In [14]:
clf.fit(x, y)

KeyError: "None of [Index([(8209, 17588, 25938, 14269, 8517, 25798, 30979, 28442, 11185, 20106, 13329, 14279, 27781, 14505, 17431, 24632, 12553, 9122, 5545, 16306, 14905, 16768, 18025, 225, 11379, 25692, 31219, 19794, 8382, 11756, 14859, 29817, 14246, 3931, 18911, 17425, 4224, 12339, 6729, 21812, 22665, 10159, 14147, 9632, 20064, 19999, 11240, 22850, 27496, 1219, 31361, 30935, 13089, 3510, 455, 20428, 29361, 17444, 4252, 28317, 27492, 6622, 20278, 26774, 29033, 29098, 16051, 4303, 26787, 22523, 2455, 14611, 25673, 7544, 26780, 26486, 31115, 1683, 15265, 22987, 12137, 17404, 3120, 3707, 5615, 30357, 16638, 3301, 9913, 10870, 25404, 2709, 26353, 2390, 27565, 2852, 6287, 1115, 19728, 16294, ...)], dtype='object')] are in the [columns]"

## Testing

In [ ]:
y_pred = clf.predict(test.iloc[:,:-1])

## Evaluating

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score



accuracy = accuracy_score(y_true=test.iloc[:, -1], y_pred=y_pred)
roc = roc_auc_score(test.iloc[:, -1], y_pred)
pre = precision_score(y_true=test.iloc[:, -1], y_pred=y_pred)
rec = recall_score(y_true=test.iloc[:, -1], y_pred=y_pred)


print("Metrics: \n Acc: {acc:.2f}, ROC: {roc:.2f}\n PRE: {pre:.2f}, REC: {rec:.2f}".format(acc=accuracy,roc=roc,pre=pre,rec=rec))


Metrics: 
 Acc: 1.00, ROC: 1.00
 PRE: 1.00, REC: 1.00


## SC graph

In [ ]:
# accs = []
# rocs = []
# pres = []
# recs = []


# for rs in range(100):
#     train, test = train_test_split(df, test_size=0.2, random_state=rs)

#     clf = SVC(random_state=rs)

#     x = cudf.DataFrame(train.iloc[:, :-1])
#     y = cupy.array(train.iloc[:, -1])

#     clf.fit(x, y)
#     y_pred = clf.predict(test.iloc[:, :-1])

#     accuracy = accuracy_score(y_true=test.iloc[:, -1], y_pred=y_pred)
#     accs.append(accuracy)
#     roc = roc_auc_score(test.iloc[:, -1], y_pred)
#     rocs.append(roc)
#     pre = precision_score(y_true=test.iloc[:, -1], y_pred=y_pred)
#     pres.append(pre)
#     rec = recall_score(y_true=test.iloc[:, -1], y_pred=y_pred)
#     recs.append(rec)

#     print("{i}/100".format(i=rs+1))

In [ ]:


# # import matplotlib.pyplot as plt


# # plt.stairs(accs)

# # npAccs = np.array(accs)
# # npRocs = np.array(rocs)
# # npPres = np.array(pres)
# # npRecs = np.array(recs)

# def min_max(name,array):
#     a = np.array(array)
#     print("{name}: ({min:.6f},{max:.1f}|{mean:.6f})".format(
#         name=name, min=np.min(a), max=np.max(a), mean=np.mean(a)))

# min_max("Acc",accs)
# min_max("Roc", rocs)
# min_max("Pre", pres)
# min_max("Rec", recs)


Acc: (0.998750,1.0|0.999829)
Roc: (0.998751,1.0|0.999829)
Pre: (0.997504,1.0|0.999832)
Rec: (0.997532,1.0|0.999826)
